In [1]:
import io
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import io

Modelo Er

<img src="er.png">

In [2]:
conn = sqlite3.connect('trabalho.db', timeout=10)
c = conn.cursor()

In [3]:
dataset = pd.read_csv("situacao-academica-discentes-graduacao-2020-ufpe.csv")
dataset = dataset.fillna("NULL") 

C:\Users\Franc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (34,35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
dataset.head()

,NU_MATRICULA,DT_NASCIMENTO,PERIODO_INGRESSO,TIPO_INGRESSO,PERIODO_LETIVO,CAMPUS,CD_INEP,CURSO,CARGA_HORARIA_CURSO,TURNO,...,AUTISMO,SINDR_ASPERGER,SINDR_RETT,TRANST_DESINT,ALT_HAB_SUPERD,MOB_ACAD,TIPO_MOB_ACAD,IES_DEST,TIPO_MOB_ACAD_INTERNAC,PAIS_DEST
0,200102627,02/08/1977,2001.1,SUB-JUDICE,2020.1,REITOR JOAQUIM AMAZONAS,13590,GEOLOGIA,3480,INTEGRAL,...,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NULL,NULL,NULL,NULL
1,200306046,09/08/1975,2003.1,ISOLADO,2020.1,REITOR JOAQUIM AMAZONAS,0,ISOLADAS,0,INTEGRAL,...,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NULL,NULL,NULL,NULL
2,200306372,01/10/1980,2003.2,ISOLADO,2020.1,REITOR JOAQUIM AMAZONAS,0,ISOLADAS,0,INTEGRAL,...,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NULL,NULL,NULL,NULL
3,200406008,07/11/1976,2004.2,ISOLADO,2020.1,REITOR JOAQUIM AMAZONAS,0,ISOLADAS,0,INTEGRAL,...,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NULL,NULL,NULL,NULL
4,200644864,07/07/1984,2008.1,VESTIBULAR,2020.1,REITOR JOAQUIM AMAZONAS,52249,GEOGRAFIA - BACHARELADO,2370,TARDE,...,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,NULL,NULL,NULL,NULL


Declaração

In [5]:
#Criar a tabela FORMA_INGRESSO
c.execute('''CREATE TABLE FORMA_INGRESSO
             (COD_INGRESSO         INT NOT NULL,
              ANO_INGRESSO         INT NOT NULL,
              SEMESTRE_INGRESSO    INT NOT NULL,
              TIPO_INGRESSO        VARCHAR(20) NOT NULL,
              COTAS                VARCHAR(20) NOT NULL,
              PRIMARY KEY(COD_INGRESSO)    
             )''')
    

In [6]:
#Criar a tabela DESTINO
c.execute('''CREATE TABLE DESTINO
             (COD_DESTINO         INT NOT NULL,
              INSTITUIÇÃO          VARCHAR(20),
              PAÍS                 VARCHAR(20),
              PRIMARY KEY(COD_DESTINO)    
             )''')

In [7]:
#Criar a tabela CURSO
c.execute('''CREATE TABLE CURSO
             (CD_INEP                 INT NOT NULL,
              CURSO                   VARCHAR(50) NOT NULL,
              CAMPOS                  VARCHAR(50) NOT NULL,
              CARGA_HORARIO_CURSO     INT,
              PRIMARY KEY(CD_INEP)    
             )''')

In [8]:
#Criar a tabela LOCALIDADE
c.execute('''CREATE TABLE LOCALIDADE
             (COD_LOCAL               INT NOT NULL,
              CIDADE                  VARCHAR(100) NOT NULL,
              UF                      VARCHAR(100) NOT NULL,
              PAIS                    VARCHAR(100) NOT NULL,
              PRIMARY KEY(COD_LOCAL)    
             )''')

In [9]:
#Criar a tabela discente
c.execute('''CREATE TABLE DISCENTE
             (NU_MATRICULA         INT NOT NULL,
              SEXO                 CHAR(1),
              ESCOLA_DE_ORIGEM     VARCHAR(20),
              COR                  VARCHAR(20),
              COD_INGRESSO         INT NOT NULL,
              COD_DESTINO          INT,
              COD_LOCAL            INT NOT NULL,
              DATA_NASC            VARCHAR(100),
              PRIMARY KEY(NU_MATRICULA),
              
              FOREIGN KEY(COD_INGRESSO) REFERENCES FORMA_INGRESSO(COD_INGRESSO) ON DELETE SET NULL,
              FOREIGN KEY(COD_DESTINO) REFERENCES DESTINO(COD_DESTINO) ON DELETE SET NULL,
              FOREIGN KEY(COD_LOCAL) REFERENCES LOCALIDADE(COD_LOCAL) ON DELETE SET NULL
             )''')
    

In [10]:
#Criar a tabela DEFICIENCIA
c.execute('''CREATE TABLE DEFICIENCIA
             (COD_DEFICIENCIA         INT NOT NULL,
              NOME                    VARCHAR(100) NOT NULL,
              MULTIPLO                INT NOT NULL,
              TIPO                    VARCHAR(100) NOT NULL,
              PRIMARY KEY(COD_DEFICIENCIA)    
             )''')

In [11]:
#Criar a tabela POSSUI
c.execute('''CREATE TABLE POSSUI
             (NU_MATRICULA         INT NOT NULL,
              COD_DEFICIENCIA      INT NOT NULL,
              PRIMARY KEY(NU_MATRICULA,COD_DEFICIENCIA),
              FOREIGN KEY(NU_MATRICULA) REFERENCES DISCENTE(NU_MATRICULA) ON DELETE CASCADE,
              FOREIGN KEY(COD_DEFICIENCIA) REFERENCES DEFICIENCIA(COD_DEFICIENCIA) ON DELETE CASCADE
             )''')

In [12]:
#Criar a tabela Cursa
c.execute('''CREATE TABLE CURSA
             (CD_CURSA             INT NOT NULL,
              NU_MATRICULA         INT NOT NULL,
              CD_INEP              INT NOT NULL,
              MODALIDADE           VARCHAR(100),
              TURNO                VARCHAR(100),
              PERIODO_LETIVO       VARCHAR(100),
              PRIMARY KEY(CD_CURSA,NU_MATRICULA,CD_INEP),
              FOREIGN KEY(NU_MATRICULA) REFERENCES DISCENTE(NU_MATRICULA) ON DELETE CASCADE,
              FOREIGN KEY(CD_INEP) REFERENCES CURSO(CD_INEP) ON DELETE CASCADE
             )''')

Populando

In [13]:
#Popular FORMA_INGRESSO

comb = []
for x in range(dataset.shape[0]):
    discente = dataset.loc[x]
    
    COD_INGRESSO = len(comb)
    
    periodo = str(discente["PERIODO_INGRESSO"]).split(".")
    
    ANO_INGRESSO = int(periodo[0])
    
    SEMESTRE_INGRESSO = int(periodo[1])
    
    TIPO_INGRESSO = discente["TIPO_INGRESSO"]
    
    COTAS = discente["COTAS"]
    
    querry = """INSERT INTO FORMA_INGRESSO
              VALUES ({},{},{},'{}', '{}') """.format(COD_INGRESSO,ANO_INGRESSO,SEMESTRE_INGRESSO,TIPO_INGRESSO,COTAS)
    
    lista = [ANO_INGRESSO,SEMESTRE_INGRESSO,TIPO_INGRESSO,COTAS]
    
    if comb.count(lista) == 0:
        comb.append(lista)
    else:
        continue
    
    
    #Tratar erros do dataset
    try:
        c.execute(querry)
    except:
        continue


In [14]:
#Popular DESTINO

comb = []
for x in range(dataset.shape[0]):
    discente = dataset.loc[x]
    
    COD_DESTINO = len(comb)
    
    INSTITUICAO = discente["IES_DEST"]
    
    PAIS_DEST = discente["PAIS_DEST"]
    
    #Caso em que ambos são NULL
    if INSTITUICAO == PAIS_DEST:
        continue
    
    querry = """INSERT INTO DESTINO
              VALUES ({},'{}','{}') """.format(COD_DESTINO,INSTITUICAO,PAIS_DEST)
    
    lista = [INSTITUICAO,PAIS_DEST]
    
    if comb.count(lista) == 0:
        comb.append(lista)
    else:
        continue
        
    #Tratar erros do dataset
    try:
        c.execute(querry)
    except:
        continue


In [15]:
#Popular CURSO

comb = []
for x in range(dataset.shape[0]):
    discente = dataset.loc[x]
    
    CD_INEP = discente["CD_INEP"]
    
    if comb.count(CD_INEP) != 0:
        continue
        
    CURSO = discente["CURSO"]
    
    CAMPUS = discente["CAMPUS"]
    
    CARGA_HORARIA_CURSO = discente["CARGA_HORARIA_CURSO"]
    
    
    querry = """INSERT INTO CURSO
              VALUES ({},'{}','{}', {}) """.format(CD_INEP,CURSO,CAMPUS,CARGA_HORARIA_CURSO)
    

        
    #Tratar erros do dataset
    try:
        c.execute(querry)
    except:
        continue


In [16]:
#Popular LOCALIDADE

comb = []
for x in range(dataset.shape[0]):
    discente = dataset.loc[x]
    
    COD_LOCAL = len(comb)
    
    CIDADE = discente["CIDADE_NAT"]
    
    UF = discente["UF_NAT"]
    
    CIDADE = discente["CIDADE_NAT"]
    
    if CIDADE == "DIAS D'AVILA":
        CIDADE = "DIAS D AVILA"
    
    PAIS = discente["NACIONALIDADE"]
    
    #Tratar NULL
    if CIDADE == UF:
        continue
    
    
    querry = """INSERT INTO LOCALIDADE
              VALUES ({},'{}','{}','{}') """.format(COD_LOCAL,CIDADE,UF,PAIS)
    
    lista = [CIDADE,UF,CIDADE,PAIS]
    
    if comb.count(lista) == 0:
        comb.append(lista)
    else:
        continue
        
        
    #Tratar erros do dataset
    try:
        c.execute(querry)
    except:
        continue

In [17]:
#Popular discente

for x in range(dataset.shape[0]):
    discente = dataset.loc[x]
    
    num_matricula = discente["NU_MATRICULA"]
    sexo = discente["SEXO"]
    escola = discente["ESCOLA_PUBLICA"]
    cor = discente["COR"]
    data_nasc = discente["DT_NASCIMENTO"]
    
    if escola == "NÃO INFORMADO":
        escola = "NULL"
    elif escola == "NÃO":
        escola = "PARTICULAR"
    else:
        escola = "PÚBLICA"
    if cor == "NÃO DECLARADO":
        cor = "NULL"
        
    #Pegar COD_INGRESSO
    periodo = str(discente["PERIODO_INGRESSO"]).split(".")
    
    ANO_INGRESSO = int(periodo[0])
    
    SEMESTRE_INGRESSO = int(periodo[1])
    
    TIPO_INGRESSO = discente["TIPO_INGRESSO"]
    
    COTAS = discente["COTAS"]
    
    selecionar = """

    Select  *
    From FORMA_INGRESSO
    WHERE ANO_INGRESSO = {} AND SEMESTRE_INGRESSO = {} AND TIPO_INGRESSO = '{}' AND COTAS = '{}'
    """.format(ANO_INGRESSO, SEMESTRE_INGRESSO, TIPO_INGRESSO,COTAS)

    df = pd.read_sql_query(selecionar, conn)
    
    if df.shape[0] == 0:
        continue
    
    COD_INGRESSO = df["COD_INGRESSO"][0]
    
    #Pegar COD_DESTINO
    
    INSTITUICAO = discente["IES_DEST"]
    
    PAIS_DEST = discente["PAIS_DEST"]
    
    selecionar = """

    Select  *
    From DESTINO
    Where INSTITUIÇÃO = '{}' AND PAÍS = '{}'
    """.format(INSTITUICAO,PAIS_DEST)
    
    df = pd.read_sql_query(selecionar, conn)
    
    if INSTITUICAO == PAIS_DEST:
        COD_DESTINO = "NULL"
    else:
        COD_DESTINO = df["COD_DESTINO"][0]
    
    
    #pegar COD_LOCAL
    
    CIDADE = discente["CIDADE_NAT"]
    
    if CIDADE == "DIAS D'AVILA":
        CIDADE = "DIAS D AVILA"
    
    UF = discente["UF_NAT"]
        
    PAIS = discente["NACIONALIDADE"]
    
    selecionar = """

    Select  *
    From LOCALIDADE
    Where CIDADE = '{}' AND UF = '{}' AND PAIS = '{}'
    """.format(CIDADE,UF,PAIS)
    
    df = pd.read_sql_query(selecionar, conn)
    
    if df.shape[0] == 0:
        continue
    
    COD_LOCAL = df["COD_LOCAL"][0]
    
    
           
    
    querry = """INSERT INTO DISCENTE
              VALUES ({},'{}','{}','{}',
              {}, {}, {}, '{}') """.format(num_matricula,sexo,escola,cor,
                                    COD_INGRESSO,COD_DESTINO,COD_LOCAL,data_nasc)
    
    
    
    #Tratar erros do dataset
    try:
        c.execute(querry)
    except:
        continue


In [18]:
#Popular DEFICIENCIA
querry = """INSERT INTO DEFICIENCIA
            VALUES (0,'SURDEZ',0,'FISICA') """
c.execute(querry)


querry = """INSERT INTO DEFICIENCIA
            VALUES (1,'AUDITIVA',0,'FISICA') """
c.execute(querry)


querry = """INSERT INTO DEFICIENCIA
            VALUES (2,'FISICA',0,'FISICA') """
c.execute(querry)
    

querry = """INSERT INTO DEFICIENCIA
            VALUES (3,'SURDOCEGUEIRA',1,'FISICA') """
c.execute(querry)


querry = """INSERT INTO DEFICIENCIA
            VALUES (4,'MULTIPLA',1,'FISICA') """
c.execute(querry)


querry = """INSERT INTO DEFICIENCIA
            VALUES (5,'INTELECTUAL',0,'MENTAL') """
c.execute(querry)


querry = """INSERT INTO DEFICIENCIA
            VALUES (6,'AUTISMO',0,'MENTAL') """
c.execute(querry)


querry = """INSERT INTO DEFICIENCIA
            VALUES (7,'SINDR_ASPERGER',0,'MENTAL') """
c.execute(querry)


querry = """INSERT INTO DEFICIENCIA
            VALUES (8,'SINDR_RETT',0,'MENTAL') """
c.execute(querry)


querry = """INSERT INTO DEFICIENCIA
            VALUES (9,'TRANST_DESINT',0,'MENTAL') """
c.execute(querry)


querry = """INSERT INTO DEFICIENCIA
            VALUES (10,'ALT_HAB_SUPERD',0,'MENTAL') """
c.execute(querry)



In [19]:
#Populando POSSUI

deficiencias = "SURDEZ AUDITIVA FISICA SURDOCEGUEIRA MULTIPLA INTELECTUAL AUTISMO SINDR_ASPERGER SINDR_RETT TRANST_DESINT ALT_HAB_SUPERD".split()
for x in range(dataset.shape[0]):
    discente = dataset.loc[x]
    
    if discente["DEFIC"] == "SIM":
        id = 0        
        NU_MATRICULA = discente["NU_MATRICULA"]
        for y in deficiencias:
            if discente[y] == "SIM":
                querry = """INSERT INTO POSSUI
                            VALUES ({},{}) """.format(NU_MATRICULA,id)
                c.execute(querry)
            id += 1



In [20]:
#Populando CURSA

comb = []
for x in range(dataset.shape[0]):
    discente = dataset.loc[x]
    NU_MATRICULA = discente["NU_MATRICULA"]
    CD_INEP = discente["CD_INEP"]
    MODALIDADE = discente["MODALIDADE"]
    TURNO = discente["TURNO"]
    PERIODO_LETIVO = discente["PERIODO_LETIVO"]
    
    lista = [NU_MATRICULA,CD_INEP,MODALIDADE,TURNO,PERIODO_LETIVO]

    id = len(comb)
    
    if comb.count(lista) == 0:
        comb.append(lista)
    else:
        continue

    querry = """INSERT INTO CURSA
                VALUES ({},{},{},'{}','{}','{}') """.format(id,NU_MATRICULA,CD_INEP,MODALIDADE,TURNO,PERIODO_LETIVO)
    

    c.execute(querry)


Visualizar

In [21]:
query = """

Select  *
From DISCENTE
"""

df = pd.read_sql_query(query, conn)
df

,NU_MATRICULA,SEXO,ESCOLA_DE_ORIGEM,COR,COD_INGRESSO,COD_DESTINO,COD_LOCAL,DATA_NASC
0,200102627,M,NULL,NULL,0,NaN,0,02/08/1977
1,200306046,F,NULL,BRANCA,1,NaN,0,09/08/1975
2,200306372,F,NULL,BRANCA,2,NaN,0,01/10/1980
3,200406008,F,NULL,NULL,3,NaN,0,07/11/1976
4,200644864,M,NULL,NULL,4,NaN,0,07/07/1984
...,...,...,...,...,...,...,...,...
32662,200987235,F,NULL,BRANCA,196,NaN,0,05/06/2002
32663,200987238,F,NULL,BRANCA,196,NaN,34,21/08/1999
32664,200987240,F,NULL,PARDA,195,NaN,11,18/03/1995
32665,200987242,M,NULL,PARDA,197,NaN,253,25/05/1998


In [22]:
query = """

Select  *
From FORMA_INGRESSO
"""

df = pd.read_sql_query(query, conn)
df

,COD_INGRESSO,ANO_INGRESSO,SEMESTRE_INGRESSO,TIPO_INGRESSO,COTAS
0,0,2001,1,SUB-JUDICE,AC
1,1,2003,1,ISOLADO,AC
2,2,2003,2,ISOLADO,AC
3,3,2004,2,ISOLADO,AC
4,4,2008,1,VESTIBULAR,AC
...,...,...,...,...,...
200,200,2020,1,SUB-JUDICE,L6
201,201,2020,1,SUB-JUDICE,L2
202,202,2020,1,ISOLADO,AC
203,203,2020,1,MOBILIDADE ESTUDANTIL,AC


In [23]:
query = """

Select  *
From DESTINO
"""

df = pd.read_sql_query(query, conn)
df

,COD_DESTINO,INSTITUIÇÃO,PAÍS
0,0,UNIVERSITE BLAISE PASCAL,FRANÇA
1,1,PROGRAMA CIÊNCIA SEM FRONTEIRAS,ESTADOS UNIDOS DA AMERICA
2,2,CAPES/BRAFITEC,FRANÇA
3,3,Universidade Federal do Oeste do Para,BRASIL
4,4,Universidade Federal do Rio Grande do Sul,BRASIL
5,5,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASIL
6,6,UNIVERSIDADE FEDERAL DO ABC,BRASIL
7,7,UNIVERSIDADE FEDERAL DE SANTA CATARINA,BRASIL
8,8,UNIVERSIDADE FEDERAL DE MINAS GERAIS,BRASIL
9,9,UNIVERSIDADE DE COIMBRA,PORTUGAL


In [24]:
query = """

Select  *
From CURSO
"""

df = pd.read_sql_query(query, conn)
df

,CD_INEP,CURSO,CAMPOS,CARGA_HORARIO_CURSO
0,13590,GEOLOGIA,REITOR JOAQUIM AMAZONAS,3480
1,0,ISOLADAS,REITOR JOAQUIM AMAZONAS,0
2,52249,GEOGRAFIA - BACHARELADO,REITOR JOAQUIM AMAZONAS,2370
3,13585,ENGENHARIA ELETRÔNICA,REITOR JOAQUIM AMAZONAS,3660
4,13588,ENGENHARIA QUÍMICA,REITOR JOAQUIM AMAZONAS,3885
...,...,...,...,...
105,1330038,GEOGRAFIA - EAD,REITOR JOAQUIM AMAZONAS,3170
106,1321439,COMUNICAÇÃO SOCIAL - CAA,AGRESTE,3030
107,53750,MUSICA - CANTO,REITOR JOAQUIM AMAZONAS,1710
108,5000630,ABI - ENGENHARIA,REITOR JOAQUIM AMAZONAS,645


In [25]:
query = """

Select  *
From LOCALIDADE
"""

df = pd.read_sql_query(query, conn)
df

,COD_LOCAL,CIDADE,UF,PAIS
0,0,RECIFE,PE,BRA
1,1,BODOCO,PE,BRA
2,2,CARUARU,PE,BRA
3,3,JABOATAO DOS GUARARAPES,PE,BRA
4,4,OLINDA,PE,BRA
...,...,...,...,...
745,745,ALAGOINHAS,BA,BRA
746,746,OUTROS,ZZ,BRA
747,747,GAMBA,ZZ,BRA
748,748,SEME TERMINAL,ZZ,BRA


In [26]:
query = """

Select  *
From DEFICIENCIA
"""

df = pd.read_sql_query(query, conn)
df

,COD_DEFICIENCIA,NOME,MULTIPLO,TIPO
0,0,SURDEZ,0,FISICA
1,1,AUDITIVA,0,FISICA
2,2,FISICA,0,FISICA
3,3,SURDOCEGUEIRA,1,FISICA
4,4,MULTIPLA,1,FISICA
5,5,INTELECTUAL,0,MENTAL
6,6,AUTISMO,0,MENTAL
7,7,SINDR_ASPERGER,0,MENTAL
8,8,SINDR_RETT,0,MENTAL
9,9,TRANST_DESINT,0,MENTAL


In [27]:
query = """

Select *
From POSSUI
"""

df = pd.read_sql_query(query, conn)
df

,NU_MATRICULA,COD_DEFICIENCIA
0,200711907,2
1,200711973,4
2,200739198,5
3,200773305,10
4,200780001,2
...,...,...
326,200986665,1
327,200986707,2
328,200986722,1
329,200987084,1


In [28]:
query = """

Select *
From CURSA
"""

df = pd.read_sql_query(query, conn)
df

,CD_CURSA,NU_MATRICULA,CD_INEP,MODALIDADE,TURNO,PERIODO_LETIVO
0,0,200102627,13590,PRESENCIAL,INTEGRAL,2020.1
1,1,200306046,0,PRESENCIAL,INTEGRAL,2020.1
2,2,200306372,0,PRESENCIAL,INTEGRAL,2020.1
3,3,200406008,0,PRESENCIAL,INTEGRAL,2020.1
4,4,200644864,52249,PRESENCIAL,TARDE,2020.1
...,...,...,...,...,...,...
32806,32806,200987235,29483,PRESENCIAL,TARDE,2020.1
32807,32807,200987238,13611,PRESENCIAL,NOITE,2020.1
32808,32808,200987240,1189778,PRESENCIAL,INTEGRAL,2020.1
32809,32809,200987242,1189778,PRESENCIAL,INTEGRAL,2020.1


Partes finais

In [29]:
conn.commit()

In [30]:
#backup
#Referencia: http://pythonclub.com.br/gerenciando-banco-dados-sqlite3-python-parte1.html

f = io.open("trabalho.sql",'w')

for texto in conn.iterdump():
    f.write("{}\n".format(texto))

f.close()

In [31]:
conn.close()